In [2]:
import os

In [3]:
!pwd

/Users/kashifjilani/Documents/DVC/MachineLearning-MLFlow/research


In [4]:
os.chdir("../")

In [45]:
%pwd

'/Users/kashifjilani/Documents/DVC/MachineLearning-MLFlow'

In [76]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [77]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [78]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Path("config/config.yaml")):
        self.config = read_yaml(config_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        return data_ingestion_config

In [79]:
import os
import zipfile
import urllib.request as request
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [80]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [81]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-31 10:20:13,171: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-07-31 10:20:13,173: INFO: common: created directory as: artifacts]
[2023-07-31 10:20:13,175: INFO: common: created directory as: artifacts/data_ingestion]
[2023-07-31 10:20:15,838: INFO: 214553537: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 25472
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3a28b21a381d2cb2b59a08391b0b226053b9450fcd236ab32a114ebab7de40c6"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5B8A:3E9B49:BF309:1261D0:64C73D86
Accept-Ranges: bytes
Date: Mon, 31 Jul 2023 04:50:15 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830047-CCU
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1690779015.421579,VS0,VE338
Vary: Authorization,Accept-En

In [58]:
CONFIG_FILE_PATH = Path("config/config.yaml")
print(CONFIG_FILE_PATH)

config/config.yaml
